In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2(func) import *
from smile_utils import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

np.random.seed(1)

## Общий план задания

Чтобы построить нейронную сеть, надо реализовывать несколько "вспомогательных функций". Эти вспомогательные функции будут использованы в следующем задании для построения двухслойной нейронной сети и L-слойной нейронной сети.

- Инициализируйте параметры для двухслойной сети и для $L$-слойной нейронной сети.
- Реализовать модуль forward propagation (показан фиолетовым цветом на рисунке ниже).
    - Завершите  the part of a LINEAR layer's forward propagation step  (в результате чего $Z^{[l]}$).
    - Функцию активации (relu/ sigmoid).
    - Объедините предыдущие два шага в новую функцию  [LINEAR->ACTIVATION].
    - Сложите  функцию [LINEAR - >RELU] L-1 раз (для слоев с 1 по L-1) и добавьте [LINEAR->SIGMOID] в конце (для конечного слоя $L$). Это дает новую функцию L_model_forward.
- Подсчитайте потери.

- Реализовать backward propagation module (обозначенный красным цветом на рисунке ниже).
    - Complete the LINEAR part of a layer's backward propagation step.
    - Мы даем вам градиент функции активации (relu_backward/sigmoid_backward)
    - Объедините предыдущие два шага в новую [LINEAR->ACTIVATION] backward функцию.
    - Сложите  функцию [LINEAR->RELU]] Л-1 раз и добавить [LINEAR->SIGMOID] backward в новую функцию L_model_backward
- Наконец-то обновите параметры.

<img src="https://user-images.githubusercontent.com/54672403/84135621-e12bb680-aa52-11ea-8991-e16f38d67c0f.png" style="width:800px;height:500px;">

**Обратите внимание**, что для каждой forward функции существует соответствующая backward функция. Именно поэтому на каждом шаге вашего forward модуля вы будете хранить некоторые значения в кэше. Кэшированные значения полезны для вычисления градиентов. Затем в модуле обратного распространения вы будете использовать кэш для вычисления градиентов.

## Initialization

Вы напишете две вспомогательные функции, которые будут инициализировать параметры для вашей модели. Первая функция будет использоваться для инициализации параметров двухслойной модели. Во втором случае этот процесс инициализации будет обобщен на слои $L$.


###  2-layer Neural Network

Создайте и инициализируйте параметры 2-слойной нейронной сети.

**Инструкции**:
- Структура модели такова:  *LINEAR -> RELU -> LINEAR -> SIGMOID*. 
- Используйте случайную инициализацию для весовых матриц -  `np.random.randn(shape)*0.01`
- Используйте нулевую инициализацию для смещений - `np.zeros(shape)`.

In [2]:
def initialize_parameters(n_x, n_h, n_y):
    """
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    returns: parameters (dict) содержащий параметры:
                    W1 -- weight (matrix) shape (n_h, n_x)
                    b1 -- bias (vector)   shape (n_h, 1)
                    W2 -- weight (matrix) shape (n_y, n_h)
                    b2 -- bias (vector)   shape (n_y, 1)
    """
    
    np.random.seed(1)
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h,1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y,1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    parameters = {"W1": W1, "b1": b1,
                  "W2": W2, "b2": b2}
    return parameters    

In [3]:
parameters = initialize_parameters(2,2,1)
print ("\nW1 = \n", parameters["W1"])
print ("\nb1 =", *parameters["b1"])
print ("W2 = ", parameters["W2"])
print ("b2 = ", parameters["b2"])


W1 = 
 [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]

b1 = [0.] [0.]
W2 =  [[ 0.00865408 -0.02301539]]
b2 =  [[0.]]


###  L-layer Neural Network

Инициализация для более глубокой L-слойной нейронной сети является более сложной, поскольку существует гораздо больше весовых матриц и векторов смещения. При заполнении `initialize_parameters_deep` вы должны убедиться, что ваши размеры совпадают между каждым слоем. Напомним, что $n^{[l]}$ - это количество единиц измерения в слое $l$. Так, например, если размер наших входных данных $X$ равен $(12288, 209)$ (с примерами $m=209$), то:

<table style="width:100%">


    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
   
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    
   <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    
    
   <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>

</table>

Помните, что когда мы вычисляем $W X + b$ в python, он выполняет carries out broadcasting. Например, если:

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Тогда $WX + b$ будет:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

реализация инициализации для нейронной сети L-уровня.

**Инструкции**:
- Структура модели *[LINEAR -> RELU] $ \times$ (L-1) -> LINEAR -> SIGMOID*. То есть она имеет слои $L-1$, использующие функцию активации ReLU, за которой следует выходной слой с сигмовидной функцией активации.
- Используйте случайную инициализацию для весовых матриц  `np.random.rand(shape) * 0.01`.
- Используйте инициализацию нулей для смещений  `np.zeros(shape)`.
- Мы будем хранить $n^{[l]}$, количество единиц измерения в разных слоях, в переменной `layer_dims`. Например, `layer_dims` для "Planar Data classification model" был бы [2,4,1]: там было два входа, один скрытый слой с 4 скрытыми единицами и выходной слой с 1 выходной единицей. Таким образом, значит и `W1` форма была (4,2), `В1` был (4,1), `В2` был (1,4) и `В2` был (1,1). Теперь вы обобщите это до$ L $ слоев!
- Вот реализация для $L=1$ (однослойная нейронная сеть). Это должно вдохновить вас на реализацию общего случая (L-слойная нейронная сеть).
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [4]:
def initialize_parameters_deep(layer_dims):
    """
    layer_dims (list) cодержит размеры каждого слоя в нашей сети
    
    returns: parameters (dict) содержащий параметры "W1", "b1", ..., "WL", "bL":
                    Wl -- weight (matrix) shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias (vector)   shape (layer_dims[l], 1)
    """
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims) # количество слоев в сети

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))
    return parameters

In [5]:
parameters = initialize_parameters_deep([5,4,3])
print ("\nW1 = \n", parameters["W1"])
print ("\nb1 =", *parameters["b1"])
print ("W2 = \n", parameters["W2"])
print ("b2 = ",*parameters["b2"])


W1 = 
 [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]

b1 = [0.] [0.] [0.] [0.]
W2 = 
 [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 =  [0.] [0.] [0.]


##  Forward propagation module

### Linear Forward 
Теперь, когда вы инициализировали свои параметры, вы сделаете forward propagation module. Вы начнете с реализации некоторых основных функций, которые вы будете использовать позже при реализации модели. Вы будете выполнять три функции в таком порядке:

- LINEAR
- LINEAR -> ACTIVATION where ACTIVATION will be either ReLU or Sigmoid. 
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (whole model)

Linear forward module векторизованный по всем примерам) вычисляет следующие уравнения:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

где $A^{[0]} = X$. 


постройте линейную часть прямого распространения.

In [6]:
def linear_forward(A, W, b):
    """
    линейная часть forward propagation.

    A -- активации с предыдущего слоя (или input data): (size of previous layer, number of examples)
    W -- weights (matrix): numpy array shape (size of current layer, size of previous layer)
    b -- bias (vector), numpy array    shape (size of the current layer, 1)

    returns: 
        Z -- входной сигнал функции активации, также называемый параметром предварительной активации
        cache (dict) содержащий "A", " W " и "b"; хранится для эффективного вычисления backward
    """
    Z = np.dot(W,A) + b
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    return Z, cache

In [7]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


### Linear-Activation Forward

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. Мы снабдили вас функцией `sigmoid`. Эта функция возвращает два элемента: значение активации `A` и `кэш`, содержащий `Z` (это то, что мы будем подавать в соответствующую backward функцию).
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: математическая формула для ReLu $A = RELU(Z) = max(0, Z)$. Мы предоставили вам функцию `relu`. Эта функция возвращает два элемента: значение активации `A` и `кэш`, содержащий `Z` (это то, что мы будем подавать в соответствующую backward функцию). 
``` python
A, activation_cache = relu(Z)
```

Для большего удобства вы сгруппируете две функции (Линейная и активационная) в одну функцию (LINEAR->ACTIVATION). Таким образом, вы реализуете функцию, которая выполняет LINEAR forward шаг, за которым следует шаг ACTIVATION forward

**Упражнение**: реализуйте прямое распространение слоя *LINEAR->ACTIVATION*. Математическое соотношение таково: $A^{[l]} = g (Z^{[l]}) = g (W^{[l]}A^{[l-1]} +b^{[l]})$, где активация "g" может быть sigmoid() или relu().

In [8]:
def linear_activation_forward(A_prev, W, b, activation):
    """
    forward propagation для LINEAR->ACTIVATION layer

    A_prev -- активации с предыдущего слоя (или input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array  shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array     shape (size of the current layer, 1)
    activation -- {"sigmoid"|"relu"} активация, которая будет использоваться в этом слое 

    returns:
        A  -- вывод функции активации, также называемой значением после активации
        cache -- (dict) содержащий "linear_cache" и " activation_cache";
            сохраненный для эффективного вычисления backward 
    """
    
    # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z)
     
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)
    return A, cache

In [9]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " , *A)

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " , *A)

With sigmoid: A =  [0.96890023 0.11013289]
With ReLU: A =  [3.43896131 0.        ]


**Примечание**: в глубоком обучении вычисление "[линейная->активация] " считается одним слоем в нейронной сети, а не двумя слоями.

### L-Layer Model 

Для еще большего удобства при реализации нейронной сети уровня $L$вам понадобится функция, которая повторяет предыдущую (`linear_activation_forward` с RELU) $L-1$ раз, а затем следует за ней с одним `linear_activation_forward` с СИГМОИДОМ.

<img src="https://user-images.githubusercontent.com/54672403/84140241-f8ba6d80-aa59-11ea-813e-fb94dc459a06.png" style="width:600px;height:300px;">
<caption><center>  *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model</center></caption><br>

**Упражнение**: реализуйте прямое распространение приведенной выше модели.

**Инструкция**: В приведенном ниже коде переменная `AL` будет обозначать $A^{[L]} = \sigma(Z^{[L]}) = \sigma (W^{[L]} A^{[L-1]} + b^{[L]})$. (Это иногда также называется `Yhat`, т.е. это $\hat{Y}$.)


- Используйте функции, которые вы ранее написали
- Используйте цикл for для [LINEAR->RELU] (L-1) раз
- Не забывайте следить за cach в списке "caches".

In [10]:
def L_model_forward(X, parameters):
    """
    Прямое распространение для [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID 
    
    X -- (matrix) shape (input size, number of examples)
    parameters -- вывод initialize_parameters_deep()
    
    returns:
        AL -- последнее значение post-activation
        caches (list) --
            каждый cache из linear_relu_forward() (there are L-1 of them, indexed from 0 to L-2)
                   cache из linear_sigmoid_forward() (there is one, indexed L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2 # количество слоев в нейронной сети
    
    # [LINEAR -> RELU]*(L-1)
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev,parameters['W' + str(l)], parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
    
    # LINEAR -> SIGMOID
    AL, cache = linear_activation_forward(A,parameters['W' + str(L)],parameters['b' + str(L)],activation = "sigmoid")
    caches.append(cache)
    
    assert(AL.shape == (1,X.shape[1]))
    return AL, caches

In [11]:
X, parameters = L_model_forward_test_case()
AL, caches = L_model_forward(X, parameters)
print("AL = ", *AL)
print("Length of caches list = ", len(caches))

AL =  [0.17007265 0.2524272 ]
Length of caches list =  2


Отлично! Теперь у вас есть полное forward propagation, которое принимает вход X и выводит вектор строки $A^{[L]}$, содержащий ваши предсказания. Он также записывает все промежуточные значения в "кэши". Используя $A^{[L]}$, вы можете вычислить стоимость ваших прогнозов.

## Cost function

Теперь вы будете осуществлять прямое и обратное распространение. Вам нужно вычислить стоимость, потому что вы хотите проверить, действительно ли ваша модель учится.

**Упражнение**: вычислите стоимость кросс-энтропии $J$, используя следующую формулу:$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right))$$


In [12]:
def compute_cost(AL, Y):
    """
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- (vector) {0:non-cat, 1: cat}, shape (1, number of examples)

    returns: cost - cross-entropy cost
    """
    
    m = Y.shape[1]

    # потери от aL и Y.
    cost = -1 / m * np.sum(Y * np.log(AL) + (1-Y) * np.log(1-AL),axis=1,keepdims=True)
    
    cost = np.squeeze(cost) 
    assert(cost.shape == ())
    return cost

In [13]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


## Backward propagation module

Так же, как и при прямом распространении, вы будете реализовывать вспомогательные функции для обратного распространения. Помните, что обратное распространение используется для вычисления градиента функции потерь по отношению к параметрам.

**Reminder**: 
<img src="https://user-images.githubusercontent.com/54672403/84146163-e8a78b80-aa63-11ea-9c1a-140641ba94fa.png" style="width:650px;height:250px;">
<caption><center> Forward and Backward propagation для *LINEAR->RELU->LINEAR->SIGMOID* <br> *Фиолетовые блоки представляют собой прямое распространение, а красные блоки-обратное распространение.*  </center></caption>


>цепное правило исчисления может быть использовано для получения производной от потери $\mathcal{L}$ по отношению к $z^{[1]}$ в 2-слойной сети следующим образом:

$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

Чтобы вычислить градиент $dW^{[1]} = \frac {\partial L}{\partial W^{[1]}}$, вы используете предыдущее цепное правило и делаете $dW^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]}} {\partial W^{[1]}}$. Во время обратного распространения, на каждом шаге вы умножаете свой текущий градиент на градиент, соответствующий определенному слою, чтобы получить нужный вам градиент.

Эквивалентно, чтобы вычислить градиент $db^{[1]} = \frac {\partial L}{\partial b^{[1]}}$, вы используете предыдущее правило цепочки и делаете $db^{[1]} = dz^{[1]} \times \frac{\partial z^{[1]}} {\partial b^{[1]}}$.

Вот почему мы говорим о **обратном распространении**.


Теперь, подобно прямому распространению, вы собираетесь построить обратное распространение в три этапа:
- Линейная обратная связь (LINEAR backward)
- LINEAR -> ACTIVATION backward, где активация вычисляет производную либо от ReLU, либо от сигмовидной активации
-  [LINEAR -> RELU]  $\times$ (L-1) -> LINEAR -> SIGMOID backward(вся модель)

### Linear backward

Для слоя $l$ линейная часть равна: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (после чего следует активация).

Предположим, что вы уже вычислили производную $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$.Вы хотите получить $(dW^{[l]}, db^{[l]} dA^{[l-1]})$.

<img src="https://user-images.githubusercontent.com/54672403/84146815-f1e52800-aa64-11ea-9aff-b80e587be556.png" style="width:250px;height:300px;">

Tри выхода $(dW^{[l]}, db^{[l]}, dA^{[l]})$ вычисляются с использованием входных данных $dZ^{[l]}$.Вот необходимые вам формулы:
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


In [14]:
def linear_backward(dZ, cache):
    """
    Линейная часть обратного распространения для одного слоя (слой l)

    dZ -- градиент стоимости по отношению к линейному выходу (текущего слоя l)
    cache -- кортеж значений (A_prev, W, b), поступающих из прямого распространения в текущем слое

    returns:
        dA_prev - градиент стоимости по отношению к активации (предыдущего слоя l-1), такой же формы, как и A_prev
        db - градиент стоимости по отношению к b (текущий слой l), такой же формы, как и b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = 1 / m * np.dot(dZ ,A_prev.T)
    db = 1 / m * np.sum(dZ,axis = 1, keepdims=True)
    dA_prev = np.dot(W.T,dZ) 
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    return dA_prev, dW, db

In [15]:
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = \n", dA_prev)
print ("\ndW = ", *dW)
print ("db = ", *db)

dA_prev = 
 [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]

dW =  [-0.10076895  1.40685096  1.64992505]
db =  [0.50629448]


###  Linear-Activation backward

Затем вы создадите функцию, которая объединяет две вспомогательные функции: **`linear_backward`** и обратный шаг для активации **`linear_activation_backward'**.

Чтобы помочь вам реализовать ' linear_activation_backward`, мы предоставили две обратные функции:
- **`sigmoid_backward`**: реализует обратное распространение для сигмовидной единицы.
```python
dZ = sigmoid_backward(dA, activation_cache)
```

- **`relu_backward`**: Реализует обратное распространение для блока RELU.
```python
dZ = relu_backward(dA, activation_cache)
```

Если $g(.)$ - это функция активации,
`sigmoid_backward` и `relu_backward` вычислят $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]})$$.  

Реализовать метод обратного распространения ошибки для слоя *LINEAR->ACTIVATION*

In [16]:
def linear_activation_backward(dA, cache, activation):
    """
    метод обратного распространения ошибки для слоя *LINEAR->ACTIVATION*
    
    dA -- post-activation градиент для текущего слоя l
    cache -- кортеж значений (linear_cache, activation_cache) мы храним для эффективного вычисления обратного распространения
    activation -- {"sigmoid", "relu"} активация, которая будет использоваться в этом слое
    
    returns:
        dA_prev -- градиент стоимости по отношению к активации (предыдущего слоя l-1), такой же формы, как и A_prev
        dW -- градиент стоимости по отношению к активации (предыдущего слоя l-1), такой же формы, как и A_prev
        db -- градиент стоимости по отношению к b (текущий слой l), такой же формы, как и b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [17]:
AL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = \n", dA_prev)
print ("\ndW = ", *dW)
print ("db = ", *db)

dA_prev, dW, db = linear_activation_backward(AL, linear_activation_cache, activation = "relu")
print ("\nrelu:")
print ("dA_prev = \n", dA_prev)
print ("\ndW = ", *dW)
print ("db = ", *db)


sigmoid:
dA_prev = 
 [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]

dW =  [ 0.10266786  0.09778551 -0.01968084]
db =  [-0.05729622]

relu:
dA_prev = 
 [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]]

dW =  [ 0.44513824  0.37371418 -0.10478989]
db =  [-0.20837892]


### L-Model Backward 

Теперь вы будете реализовывать обратную функцию для всей сети. Напомним, что при реализации функции `L_model_forward` на каждой итерации вы сохраняли кэш,содержащий (X,W, b и z). В модуле обратного распространения вы будете использовать эти переменные для вычисления градиентов. Поэтому в функции `L_model_backward` вы будете перебирать все скрытые слои в обратном порядке, начиная со слоя $L$. На каждом шаге вы будете использовать кэшированные значения для слоя $l$ для обратного распространения через слой $l$. На рисунке ниже показан обратный проход.

<img src="https://user-images.githubusercontent.com/54672403/84168073-a68c4300-aa7f-11ea-9adf-7a86ff50eed1.png" style="width:450px;height:300px;">
<caption><center> Backward pass  </center></caption>

** Инициализация обратного распространения**:
Для обратного распространения через эту сеть мы знаем, что выход является,
$A^{[L]} = \sigma(Z^{[L]})$. Таким образом, ваш код должен вычислить `dAL ' $ = \frac {\partial \mathcal{L}} {\partial A^{[L]}}$.
Для этого используйте эту формулу (полученную с помощью исчисления, в котором вам не нужны глубокие знания):
```python
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # производная от стоимости по отношению к AL
```
Затем вы можете использовать этот градиент после активации `dAL`, чтобы продолжать движение назад. Как видно на Рисунке, Теперь вы можете ввести `dAL` в реализованную вами функцию LINEAR->SIGMOID backward (которая будет использовать кэшированные значения, сохраненные функцией L_model_forward). После этого вам придется использовать цикл `for` для итерации по всем другим слоям с помощью функции LINEAR->RELU backward. Вы должны хранить каждый dA, dW и db в словаре grads. Для этого используйте следующую формулу :

$$grads["dW" + str(l)] = dW^{[l]} $$
Например, для $l=3$ будет сохранено $dW^{[l]}$ в  `grads["dW3"]`.

**реализация обратного распространения для *[LINEAR->RELU] $\times$(L-1) -> LINEAR -> SIGMOID* model.**

In [18]:
def L_model_backward(AL, Y, caches):
    """
    backward propagation для [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID 
    
    AL - вероятностный вектор, выходной сигнал прямого распространения (L_model_forward())
    Y  - vector целевых переменных {0 - non-cat, 1-cat}
    caches -- список кэшов, содержащих:
        каждый кэш linear_activation_forward() с "relu" (это кэш[l], for l in range(L-1) т.e l = 0...L-2)
        кэш linear_activation_forward () с "sigmoid" (это кэш[L-1])
    
    returns: grads (dict) с градиентами
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # количество слоев
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # после этой линии Y имеет ту же форму, что и AL
    
    # Initializing backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    ## Lth layer (SIGMOID -> LINEAR) gradients. 
    # Inputs: "AL, Y, caches". 
    # Outputs: "grads["dAL"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1]
    (grads["dA" + str(L)],
     grads["dW" + str(L)],
     grads["db" + str(L)]) = linear_activation_backward(dAL, current_cache, activation = "sigmoid")
    
    for l in reversed(range(L-1)):
        ## lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 2)], caches".
        # Outputs: "grads["dA" + str(l + 1)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        current_cache = caches[l]
        (grads["dA" + str(l + 1)],
         grads["dW" + str(l + 1)],
         grads["db" + str(l + 1)]) = \
        linear_activation_backward(grads["dA" + str(l+2)], current_cache, activation = "relu")
    return grads

In [19]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)

print ("dW1 = \n", grads["dW1"])
print ("\n db1 =\n",grads["db1"])
print ("\n db1 = \n", grads["dA1"])


dW1 = 
 [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]

 db1 =
 [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]

 db1 = 
 [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


### Update Parameters

В этом разделе вы будете обновлять параметры модели, используя градиентный спуск:
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]}$$

где $\alpha$ - это скорость обучения. После вычисления обновленных параметров сохраните их в словаре параметров.

**Инструкции**:
Обновите параметры с помощью градиентного спуска на каждом $W^{[l]}$ и $b^{[l]}$ для $l = 1, 2, ..., L$.

In [20]:
def update_parameters(parameters, grads, learning_rate):
    """
    Обновление параметров с помощью градиентного спуска

    parameters - (dict), содержащий параметры
    grads - (dict), содержащий градиенты, (вывод L_model_backward)
    
    returns: parameters - (dict), содержащий обновленные параметры
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # количество слоев в нейронной сети

    for l in range(L):
        parameters["W" + str(l+1)] =  parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l + 1)]
        parameters["b" + str(l+1)] =  parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l + 1)]
    return parameters

In [22]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = \n", parameters["W1"])
print ("\nb1 = \n", parameters["b1"])
print ("\nW2 = ", *parameters["W2"])
print ("\nb2 = ", *parameters["b2"])


W1 = 
 [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]

b1 = 
 [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]

W2 =  [-0.55569196  0.0354055   1.32964895]

b2 =  [-0.84610769]


In [ ]:
functions required for building a deep neural net-
work